We will see how to **retrain  Xlnet** from scratch and also load pretrained xlnet  model and finetune it on our data. Their is high possibility that second approach could yield better results. We will be using GPU for training the model. Let's start the implementation.

In [0]:
import numpy as np
import os
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


We will use famous **Bag of Popcorn** dataset from Kaggle. It can be download from this Link (https://www.kaggle.com/c/word2vec-nlp-tutorial/data) Extract data to a particular directory inside Google Drive, I extracted it inside "xlnet_experiments/ExperimentData"

In [10]:
path_to_data='/content/drive/My Drive/xlnet_experiments/ExperimentData'
os.listdir(path_to_data)###

['testData.tsv', 'unlabeledTrainData.tsv', 'labeledTrainData.tsv']

#### Making data for language model

It is recommended to use data in particular format for XLNet language model training. We will convert our data to that format.

We will be needing one sentence per line and blank line means end of document. We will be using entire train and test data to make that file.

In [14]:
train_feat_df = pd.read_csv(path_to_data+'/unlabeledTrainData.tsv',sep="\t",error_bad_lines=False)
test_feat_df  = pd.read_csv(path_to_data+'/testData.tsv',sep="\t",error_bad_lines=False) 
# test_feat_df  = pd.read_csv('ExperimentData/labeledTrainData.tsv')
train_feat_df=train_feat_df[['review']]
test_feat_df=test_feat_df[['review']]
full_df = pd.concat([train_feat_df,test_feat_df])
full_df.review = full_df.review.str.lower()
changed_text=full_df.review.apply(lambda x:x+"\n"+"\n")
open(os.path.join(path_to_data,'data_lm.txt'), "w").write(''.join(changed_text))

b'Skipping line 43043: expected 2 fields, saw 3\n'


99187855

We merged train and test reviews and combined them in the desired format to write in **data_lm.txt**

In [17]:
print (os.listdir(path_to_data))

['testData.tsv', 'unlabeledTrainData.tsv', 'labeledTrainData.tsv', 'data_lm.txt']


In [7]:
##clone xlnet official repository
!git clone https://github.com/zihangdai/xlnet

fatal: destination path 'xlnet' already exists and is not an empty directory.


### Using sentence piece model from Google to tokenize text
We will be using  Google's **sentencepiece** model to tokenize the text. 

In [15]:
!pip install sentencepiece

     |████████████████████████████████| 1.0MB 9.2MB/s 


In [32]:
path_to_data

'/content/drive/My Drive/xlnet_experiments/ExperimentData'

In [29]:
curr_dir=os.getcwd()

/bin/bash: line 0: cd: path_to_data: No such file or directory


In [36]:
curr_dir

'/content'

In [60]:
%cd '/content/drive/My Drive/xlnet_experiments/ExperimentData'

/content/drive/My Drive/xlnet_experiments/ExperimentData


We will be keeping vocab_size to 12000 and unigram model.

In [37]:
# spm_train --input=data_lm.txt --model_prefix=sp10m.cased.v3 
# --vocab_size=12000 --character_coverage=0.99995 --model_type=unigram
import sentencepiece as spm
spm.SentencePieceTrainer.Train('--input=data_lm.txt  --vocab_size=12000 --model_prefix=sp10m.cased.v3 --character_coverage=0.99995 --model_type=unigram')


True

In [40]:
os.listdir(path_to_data)##now sentence piece model has been saved

['testData.tsv',
 'unlabeledTrainData.tsv',
 'labeledTrainData.tsv',
 'data_lm.txt',
 'sp10m.cased.v3.model',
 'sp10m.cased.v3.vocab']

In [73]:
%cd '/content'

/content


In [74]:
%cd '/content/xlnet'

/content/xlnet


In [77]:
print (os.listdir(path_to_data))

['testData.tsv', 'unlabeledTrainData.tsv', 'labeledTrainData.tsv', 'data_lm.txt', 'sp10m.cased.v3.model', 'sp10m.cased.v3.vocab']


In [0]:
##We will be saving tf records in a folder named experimenttfrecords

In [86]:
!python data_utils.py --use_tpu=False --bsz_per_host=8 --num_core_per_host=1 --uncased=False --seq_len=512 --reuse_len=256 --input_glob="/content/drive/My Drive/xlnet_experiments/ExperimentData/data_lm.txt" --save_dir="/content/drive/My Drive/experimenttfrecords" --sp_path="/content/drive/My Drive/xlnet_experiments/ExperimentData/sp10m.cased.v3.model" --mask_alpha=6 --mask_beta=1 --num_predict=85 --bi_data=False





W0123 08:47:10.300482 140366975461248 module_wrapper.py:139] From data_utils.py:181: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.


W0123 08:47:10.301253 140366975461248 module_wrapper.py:139] From data_utils.py:186: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0123 08:47:10.303020 140366975461248 module_wrapper.py:139] From data_utils.py:206: The name tf.gfile.Open is deprecated. Please use tf.io.gfile.GFile instead.


W0123 08:47:10.305681 140366975461248 module_wrapper.py:139] From data_utils.py:210: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.


W0123 08:47:10.307718 140366975461248 module_wrapper.py:139] From data_utils.py:211: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

INFO:tensorflow:Use glob: /content/drive/My Drive/xlnet_experiments/ExperimentData/data_lm.txt
I0123 08:47:10.307862 140366975461248 data_utils.py:211] Use glob: /

Now, it's time to train the model.

In [87]:
os.getcwd()

'/content/xlnet'

In [88]:
!wget https://storage.googleapis.com/xlnet/released_models/cased_L-12_H-768_A-12.zip

--2020-01-23 08:56:03--  https://storage.googleapis.com/xlnet/released_models/cased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.143.128, 2a00:1450:4013:c04::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.143.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 433638019 (414M) [application/zip]
Saving to: ‘cased_L-12_H-768_A-12.zip’

cased_L-12_H-768_A- 100%[===================>] 413.55M  59.4MB/s    in 7.0s    

2020-01-23 08:56:11 (59.4 MB/s) - ‘cased_L-12_H-768_A-12.zip’ saved [433638019/433638019]



In [89]:
!unzip cased_L-12_H-768_A-12.zip

Archive:  cased_L-12_H-768_A-12.zip
   creating: xlnet_cased_L-12_H-768_A-12/
  inflating: xlnet_cased_L-12_H-768_A-12/xlnet_model.ckpt.index  
  inflating: xlnet_cased_L-12_H-768_A-12/xlnet_model.ckpt.data-00000-of-00001  
  inflating: xlnet_cased_L-12_H-768_A-12/spiece.model  
  inflating: xlnet_cased_L-12_H-768_A-12/xlnet_model.ckpt.meta  
  inflating: xlnet_cased_L-12_H-768_A-12/xlnet_config.json  


###  We will be training our language model from checkpoint. Checkpoint is the model downloaded above.

We will save the newly trained model inside the directoy experimenttfrecords created above. If we omit init_checkpoint argument we will get freshly trained language model.

In [90]:
!python train_gpu.py --corpus_info_path="/content/drive/My Drive/experimenttfrecords/corpus_info.json" --model_dir="/content/drive/My Drive/experimenttfrecords/model"  --record_info_dir="/content/drive/My Drive/experimenttfrecords/tfrecords" --train_batch_size=8 --seq_len=512 --reuse_len=256 --mem_len=384 --perm_size=256 --untie_r=True --mask_alpha=6 --mask_beta=1 --num_predict=85 --train_steps=100 --iterations=2 --uncased=False --bi_data=False --save_steps=2 --d_head=64 --d_inner=3072 --d_model=768 --ff_activation=gelu --n_head=12 --n_layer=12 --n_token=32000 --init_checkpoint="/content/xlnet/xlnet_cased_L-12_H-768_A-12/xlnet_model.ckpt"




W0123 09:03:27.391689 139709386229632 module_wrapper.py:139] From train_gpu.py:315: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0123 09:03:27.391873 139709386229632 module_wrapper.py:139] From train_gpu.py:315: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0123 09:03:27.391998 139709386229632 module_wrapper.py:139] From train_gpu.py:319: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

INFO:tensorflow:n_token 32000
I0123 09:03:27.392084 139709386229632 train_gpu.py:319] n_token 32000

W0123 09:03:27.392195 139709386229632 module_wrapper.py:139] From train_gpu.py:321: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.

INFO:tensorflow:Use the following tfrecord dirs: ['/content/drive/My Drive/experimenttfrecords/tfrecords']
I0123 09:03:27.392630 139709386229632 data_utils.py:795] Use the following tfrecord dirs: ['

In [93]:
%cd '/content/'

/content


#### Now our retrained language model is ready. Cheers it's been achieved after lot of effforts. Now we will use pytorch transformers(God keep them happy) transfomer-cli  to change this Tensorflow model to pytorch model

In [94]:
!git clone https://github.com/huggingface/transformers

Cloning into 'transformers'...
remote: Enumerating objects: 154, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 18511 (delta 77), reused 143 (delta 73), pack-reused 18357
Receiving objects: 100% (18511/18511), 10.96 MiB | 8.50 MiB/s, done.
Resolving deltas: 100% (13760/13760), done.


Converting tensorflow model into pytorch model

In [0]:
# export TRANSFO_XL_CHECKPOINT_PATH='base_cased/model.ckpt'
# export TRANSFO_XL_CONFIG_PATH='base_cased/config.json '

# python transformers-cli convert --model_type xlnet  --tf_checkpoint $TRANSFO_XL_CHECKPOINT_PATH --config $TRANSFO_XL_CONFIG_PATH --pytorch_dump_output xlnet_exp_pytorch


In [96]:
%cd '/content/transformers/'

/content/transformers


In [0]:
!export TRANSFO_XL_CHECKPOINT_PATH='/content/drive/My Drive/experimenttfrecords/model/model.ckpt'
!export TRANSFO_XL_CONFIG_PATH='/content/drive/My Drive/experimenttfrecords/model/config.json'
!export PYTORCH_MODELPATH='/content/drive/My Drive/experimenttfrecords/pytorch_model'

In [0]:
!pip install transformers

In [106]:
!python transformers-cli convert --model_type xlnet  --tf_checkpoint="/content/drive/My Drive/experimenttfrecords/model/model.ckpt" --config="/content/drive/My Drive/experimenttfrecords/model/config.json" --pytorch_dump_output="/content/drive/My Drive/experimenttfrecords/pytorch_model"

INFO:transformers.modeling_xlnet:Loading TF weight beta1_power with shape []
INFO:transformers.modeling_xlnet:Loading TF weight beta2_power with shape []
INFO:transformers.modeling_xlnet:Loading TF weight global_step with shape []
INFO:transformers.modeling_xlnet:Loading TF weight model/lm_loss/bias with shape [32000]
INFO:transformers.modeling_xlnet:Loading TF weight model/lm_loss/bias/Adam with shape [32000]
INFO:transformers.modeling_xlnet:Loading TF weight model/lm_loss/bias/Adam_1 with shape [32000]
INFO:transformers.modeling_xlnet:Loading TF weight model/transformer/layer_0/ff/LayerNorm/beta with shape [768]
INFO:transformers.modeling_xlnet:Loading TF weight model/transformer/layer_0/ff/LayerNorm/beta/Adam with shape [768]
INFO:transformers.modeling_xlnet:Loading TF weight model/transformer/layer_0/ff/LayerNorm/beta/Adam_1 with shape [768]
INFO:transformers.modeling_xlnet:Loading TF weight model/transformer/layer_0/ff/LayerNorm/gamma with shape [768]
INFO:transformers.modeling_xl

#### DONE THANKYOU
You now have xlnet language model finetuned on your dataset.